Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [43]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in `1_notmnist.ipynb`.

In [44]:
pickle_file = '/home/weirgetic/Downloads/GitHub Repositories/Deep Learning/notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [45]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [46]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

In [14]:
hidden_size = 1024
batch_size = 128
l2_loss_mult = 0.001

graph = tf.Graph()

with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # layer1
  weights_1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, hidden_size]))
  biases_1 = tf.Variable(tf.zeros([hidden_size]))
  layer_1 = tf.nn.relu(tf.matmul(tf_train_dataset, weights_1) + biases_1)
    
  # layer2  
  weights_2 = tf.Variable(tf.truncated_normal([hidden_size,num_labels]))
  biases_2 = tf.Variable(tf.zeros([num_labels]))
  # Training computation.
  layer_2 = tf.matmul(layer_1,weights_2) + biases_2
    
  # calculating (Loss)' = loss + l2_regularization
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=layer_2)) + l2_loss_mult * (tf.nn.l2_loss(weights_1) + tf.nn.l2_loss(weights_2))
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss) 
       
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(layer_2)
  valid_prediction = tf.nn.softmax(
      tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weights_1) + biases_1),weights_2) + biases_2)
  test_prediction = tf.nn.softmax(
      tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, weights_1) + biases_1),weights_2) + biases_2)
              

In [15]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 721.339722
Minibatch accuracy: 9.4%
Validation accuracy: 37.2%
Minibatch loss at step 500: 200.786377
Minibatch accuracy: 80.5%
Validation accuracy: 81.1%
Minibatch loss at step 1000: 113.422081
Minibatch accuracy: 83.6%
Validation accuracy: 81.6%
Minibatch loss at step 1500: 68.392632
Minibatch accuracy: 87.5%
Validation accuracy: 83.3%
Minibatch loss at step 2000: 41.497360
Minibatch accuracy: 88.3%
Validation accuracy: 84.9%
Minibatch loss at step 2500: 25.261362
Minibatch accuracy: 87.5%
Validation accuracy: 86.6%
Minibatch loss at step 3000: 15.675457
Minibatch accuracy: 81.2%
Validation accuracy: 86.8%
Test accuracy: 92.7%


---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

I restrict my training data to batches with size : 10000

In [27]:
train_dataset = train_dataset[ :10000, :]
train_labels = train_labels[ :10000, :]
hidden_size = 1024
batch_size = 100
l2_loss_mult = 0.001

graph = tf.Graph()

with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # layer1
  weights_1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, hidden_size]))
  biases_1 = tf.Variable(tf.zeros([hidden_size]))
  layer_1 = tf.nn.relu(tf.matmul(tf_train_dataset, weights_1) + biases_1)
    
  # layer2  
  weights_2 = tf.Variable(tf.truncated_normal([hidden_size,num_labels]))
  biases_2 = tf.Variable(tf.zeros([num_labels]))
  # Training computation.
  layer_2 = tf.matmul(layer_1,weights_2) + biases_2
    
  # calculating (Loss)' = loss + l2_regularization
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=layer_2)) + l2_loss_mult * (tf.nn.l2_loss(weights_1) + tf.nn.l2_loss(weights_2))
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss) 
       
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(layer_2)
  valid_prediction = tf.nn.softmax(
      tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weights_1) + biases_1),weights_2) + biases_2)
  test_prediction = tf.nn.softmax(
      tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, weights_1) + biases_1),weights_2) + biases_2)
              

In [28]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 759.433838
Minibatch accuracy: 4.0%
Validation accuracy: 28.1%
Minibatch loss at step 500: 190.029053
Minibatch accuracy: 94.0%
Validation accuracy: 81.0%
Minibatch loss at step 1000: 115.194016
Minibatch accuracy: 100.0%
Validation accuracy: 81.9%
Minibatch loss at step 1500: 69.865540
Minibatch accuracy: 100.0%
Validation accuracy: 82.1%
Minibatch loss at step 2000: 42.376900
Minibatch accuracy: 100.0%
Validation accuracy: 82.3%
Minibatch loss at step 2500: 25.710365
Minibatch accuracy: 100.0%
Validation accuracy: 82.8%
Minibatch loss at step 3000: 15.633440
Minibatch accuracy: 100.0%
Validation accuracy: 83.7%
Test accuracy: 89.7%


---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [12]:
train_dataset = train_dataset[ :10000, :]
train_labels = train_labels[ :10000, :]
hidden_size = 1024
batch_size = 100

graph = tf.Graph()

with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # layer1
  weights_1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, hidden_size]))
  biases_1 = tf.Variable(tf.zeros([hidden_size]))
  layer_1 = tf.nn.relu(tf.matmul(tf_train_dataset, weights_1) + biases_1)
    
  #dropout  
  dropout = tf.nn.dropout(layer_1,.5)  
    
  # layer2  
  weights_2 = tf.Variable(tf.truncated_normal([hidden_size,num_labels]))
  biases_2 = tf.Variable(tf.zeros([num_labels]))
  # Training computation.
  layer_2 = tf.matmul(dropout,weights_2) + biases_2
    
  # calculating (Loss)' = loss + l2_regularization
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=layer_2))
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss) 
       
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(layer_2)
  valid_prediction = tf.nn.softmax(
      tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weights_1) + biases_1),weights_2) + biases_2)
  test_prediction = tf.nn.softmax(
      tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, weights_1) + biases_1),weights_2) + biases_2)

In [13]:
num_steps = 3001
with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 466.426910
Minibatch accuracy: 12.0%
Validation accuracy: 17.1%
Minibatch loss at step 500: 12.435490
Minibatch accuracy: 86.0%
Validation accuracy: 79.6%
Minibatch loss at step 1000: 17.513935
Minibatch accuracy: 86.0%
Validation accuracy: 80.8%
Minibatch loss at step 1500: 3.614824
Minibatch accuracy: 92.0%
Validation accuracy: 81.2%
Minibatch loss at step 2000: 2.442603
Minibatch accuracy: 91.0%
Validation accuracy: 81.9%
Minibatch loss at step 2500: 4.377119
Minibatch accuracy: 89.0%
Validation accuracy: 82.1%
Minibatch loss at step 3000: 7.136940
Minibatch accuracy: 88.0%
Validation accuracy: 81.6%
Test accuracy: 87.6%


---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


At first step i used learning decay & changed num_steps to 4096 steps :

In [68]:
hidden_size = 1024
batch_size = 128
l2_loss_mult = 0.001
decay_steps = 2048
starter_learning_rate = .5

graph = tf.Graph()

with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # layer1
  weights_1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, hidden_size]))
  biases_1 = tf.Variable(tf.zeros([hidden_size]))
  layer_1 = tf.nn.relu(tf.matmul(tf_train_dataset, weights_1) + biases_1)
    
  # layer2  
  weights_2 = tf.Variable(tf.truncated_normal([hidden_size,num_labels]))
  biases_2 = tf.Variable(tf.zeros([num_labels]))
  # Training computation.
  layer_2 = tf.matmul(layer_1,weights_2) + biases_2
    
  # calculating (Loss)' = loss + l2_regularization
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=layer_2)) + l2_loss_mult * (tf.nn.l2_loss(weights_1) + tf.nn.l2_loss(weights_2))
  
  # learning rate decay
  global_step = tf.Variable(0,trainable=False)
  learning_rate = tf.train.exponential_decay(starter_learning_rate, global_step,
                                           decay_steps, 0.98, staircase=True)
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss,global_step=global_step) 
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(layer_2)
  valid_prediction = tf.nn.softmax(
      tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weights_1) + biases_1),weights_2) + biases_2)
  test_prediction = tf.nn.softmax(
      tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, weights_1) + biases_1),weights_2) + biases_2)
              

In [69]:
num_steps = 4096

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 685.936707
Minibatch accuracy: 10.9%
Validation accuracy: 39.5%
Minibatch loss at step 500: 198.602661
Minibatch accuracy: 75.8%
Validation accuracy: 79.6%
Minibatch loss at step 1000: 114.177681
Minibatch accuracy: 83.6%
Validation accuracy: 82.7%
Minibatch loss at step 1500: 68.261932
Minibatch accuracy: 86.7%
Validation accuracy: 83.3%
Minibatch loss at step 2000: 41.397720
Minibatch accuracy: 89.1%
Validation accuracy: 84.8%
Minibatch loss at step 2500: 25.422323
Minibatch accuracy: 87.5%
Validation accuracy: 86.7%
Minibatch loss at step 3000: 15.907832
Minibatch accuracy: 80.5%
Validation accuracy: 87.0%
Minibatch loss at step 3500: 9.809486
Minibatch accuracy: 89.1%
Validation accuracy: 88.0%
Minibatch loss at step 4000: 6.364613
Minibatch accuracy: 85.9%
Validation accuracy: 88.3%
Test accuracy: 93.1%


Adding another layer to my 1-layer network :

In [ ]:
hidden_size_layer1 = 256
hidden_size_layer2 = 128
batch_size = 128
l2_loss_mult = 0.001
decay_steps = 2048
starter_learning_rate = 0.5

graph = tf.Graph()

with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # layer1
  weights_1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, hidden_size_layer1],stddev=np.sqrt(
      2.0/image_size * image_size )))
  biases_1 = tf.Variable(tf.zeros([hidden_size_layer1]))
  layer_1 = tf.nn.relu(tf.matmul(tf_train_dataset, weights_1) + biases_1)
    
  
  # layer2  
  weights_2 = tf.Variable(tf.truncated_normal([hidden_size_layer1,hidden_size_layer2],stddev=np.sqrt(
      2.0/hidden_size_layer1 )))
  biases_2 = tf.Variable(tf.zeros([hidden_size_layer2]))
  # Training computation.
  layer_2 = tf.nn.relu(tf.matmul(layer_1,weights_2) + biases_2)
  
  # layer3
  weights_3 = tf.Variable(tf.truncated_normal([hidden_size_layer2,num_labels],stddev=np.sqrt(
      2.0/hidden_size_layer2 )))
  biases_3 = tf.Variable(tf.zeros([num_labels]))
  
  logits = tf.matmul(layer_2,weights_3) + biases_3

  # calculating (Loss)' = loss + l2_regularization
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
  
  # learning rate decay
  global_step = tf.Variable(0,trainable=False)
  learning_rate = tf.train.exponential_decay(starter_learning_rate, global_step,
                                           decay_steps, 0.9, staircase=True)
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(
      loss,global_step=global_step) 
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
      tf.matmul(tf.nn.relu(tf.matmul(tf.nn.relu(
          tf.matmul(tf_valid_dataset, weights_1) + biases_1),weights_2) + biases_2),weights_3) + biases_3)
  test_prediction = tf.nn.softmax(
      tf.matmul(tf.nn.relu(tf.matmul(tf.nn.relu(
          tf.matmul(tf_test_dataset, weights_1) + biases_1),weights_2) + biases_2),weights_3) + biases_3)

In [ ]:
num_steps = 4096

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))